<a href="https://colab.research.google.com/github/Woominju/Deep-learning/blob/main/11_26_kfold_reservation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:

url = 'https://raw.githubusercontent.com/Woominju/11.26hotel-reservation/main/Hotel%20Reservations.csv'
data = pd.read_csv(url)
data.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [ ]:

encoder = LabelEncoder()


data[['type_of_meal_plan', 'room_type_reserved', 'market_segment_type']] = data[[
    'type_of_meal_plan', 'room_type_reserved', 'market_segment_type']].apply(encoder.fit_transform)


mapping = {'Not_Canceled': 0, 'Canceled': 1}
data['booking_status'] = data['booking_status'].map(mapping)

In [ ]:
data = pd.get_dummies(data, columns=['room_type_reserved'])


X = data.drop(['Booking_ID', 'booking_status'], axis=1)
y = data['booking_status']

In [ ]:
scaler = StandardScaler()
X[['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time',
   'arrival_year', 'arrival_month', 'arrival_date', 'no_of_previous_cancellations',
   'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests']] = scaler.fit_transform(
    X[['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'no_of_previous_cancellations',
       'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests']])

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

acc_scores = []
loss_scores = []

In [ ]:

for train_index, test_index in kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    model = create_model()


    history = model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)


    y_pred = (model.predict(X_test) > 0.5).astype(int)


    accuracy = accuracy_score(y_test, y_pred)
    loss = history.history['loss'][-1]
    acc_scores.append(accuracy)
    loss_scores.append(loss)

    print(f'Accuracy: {accuracy}, Loss: {loss}')


227/227 [==============================] - 0s 1ms/step
Accuracy: 0.8543073742246726, Loss: 0.3301164507865906
227/227 [==============================] - 0s 1ms/step
Accuracy: 0.8570640937284631, Loss: 0.32927119731903076
227/227 [==============================] - 0s 1ms/step
Accuracy: 0.8492074431426603, Loss: 0.32484912872314453
227/227 [==============================] - 0s 1ms/step
Accuracy: 0.847553411440386, Loss: 0.33379271626472473
227/227 [==============================] - 0s 1ms/step
Accuracy: 0.8559614059269469, Loss: 0.3287159204483032


In [ ]:

avg_acc_score = sum(acc_scores) / len(acc_scores)
avg_loss_score = sum(loss_scores) / len(loss_scores)
print(f'평균 정확도: {avg_acc_score}, 평균 Loss: {avg_loss_score}')


평균 정확도: 0.8528187456926257, 평균 Loss: 0.3293490827083588
